In [ ]:
"""
changelogs:
    change to regression model
"""

In [1]:
import torch
from torch import nn

import pathlib, tempfile
from torch.utils.tensorboard import SummaryWriter

In [2]:
model_name = 'm207'
writer = SummaryWriter(f'logs/{model_name}', filename_suffix='-model_graph.tb')

In [3]:
def calc_padding(kernel_size, padding):
    if padding == 'same':
        return kernel_size//2
    elif padding == 'valid':
        return 0

def conv1d(Ci, Co, kernel_size, stride, padding):
    module = nn.Conv1d(Ci, Co,
                       kernel_size=kernel_size,
                       stride=stride,
                       padding=calc_padding(kernel_size, padding),
                       bias=False)
    return module

def conv1d_bn_relu(Ci, Co, kernel_size, stride, padding='same'):
    module = nn.Sequential(
        conv1d(Ci, Co, kernel_size, stride, padding),
        nn.BatchNorm1d(Co),
        nn.ReLU(inplace=True)
    )
    return module

class SENetBlock(nn.Module):
    def __init__(self, Ci):
        super().__init__()
        Cm = Ci//4
        self.layers = nn.Sequential(
            nn.AdaptiveAvgPool1d(output_size=1),
            nn.Flatten(),
            nn.Linear(Ci, Cm),
            nn.ReLU(inplace=True),
            nn.Linear(Cm, Ci),
            nn.Sigmoid(),
            nn.Unflatten(1, (Ci, 1)) # reshape Ci to Ci x 1
        )
        
    def forward(self, x):
        y = self.layers(x)
        y = torch.mul(x, y)
        return y

class ResNetBlock1(nn.Module):
    '''
    residual path: 
        Conv * 3:
        kernel =  3,  3, 1
        filter = *2, *2, 1 (對應於 Ci)
    '''
    def __init__(self, Ci):
        super().__init__()
        C1 = C2 = Ci*2
        self.layers = nn.Sequential(
            conv1d_bn_relu(Ci, C1, 3, 1),
            conv1d_bn_relu(C1, C2, 3, 1),
            conv1d_bn_relu(C2, Ci, 1, 1),
            SENetBlock(Ci)
        )
    
    def forward(self, x):
        y = self.layers(x)
        y = torch.add(x, y)
        return y

class ResNetBlock2(nn.Module):
    '''
    residual path: 
        Conv * 3:
        kernel =  1,  3, 1
        filter = /2, /2, 1 (對應於 Ci)
    '''
    def __init__(self, Ci):
        super().__init__()
        C1 = C2 = Ci//2
        self.layers = nn.Sequential(
            conv1d_bn_relu(Ci, C1, 1, 1),
            conv1d_bn_relu(C1, C2, 3, 1),
            conv1d_bn_relu(C2, Ci, 1, 1),
            SENetBlock(Ci)
        )
    
    def forward(self, x):
        y = self.layers(x)
        y = torch.add(x, y)
        return y

class CSPNetBlock(nn.Module):
    def __init__(self, Ci, Co, Res_block, kernel_size, stride):
        super().__init__()
        Cn = Ci//2
        self.layers1 = conv1d_bn_relu(Ci, Cn, 1, 1)
        self.layers2 = nn.Sequential(
            conv1d_bn_relu(Ci, Cn, 1, 1),
            Res_block(Cn),
            Res_block(Cn),
            conv1d_bn_relu(Cn, Cn, 1, 1)
        )
        self.layers3 = conv1d_bn_relu(Ci, Co, kernel_size, stride)
        
    def forward(self, x):
        y1 = self.layers1(x)
        y2 = self.layers2(x)
        y = torch.cat((y1, y2), 1)
        y = self.layers3(y)
        return y

class MultiPath(nn.Module):
    def __init__(self, Ci, Co):
        super().__init__()
        C1 = Co*3//4 # Co x 3/4
        C2 = Co//4 # Co x 1/4
        self.layers1 = nn.Sequential(
            conv1d_bn_relu(Ci, C1, 19, 10, 'valid'),
            conv1d_bn_relu(C1, C1, 19, 10, 'valid')
        )
        self.layers2 = nn.Sequential(
            conv1d_bn_relu(Ci, C2, 9, 5, 'valid'),
            conv1d_bn_relu(C2, C2, 9, 5, 'valid'),
            conv1d_bn_relu(C2, C2, 7, 4, 'valid')
        )

    def forward(self, x):
        y1 = self.layers1(x)
        y2 = self.layers2(x)
        y = torch.cat((y1, y2), 1)
        return y
    
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.stem = MultiPath(1, 64)
        self.body = nn.Sequential(
            CSPNetBlock(64, 64, ResNetBlock1, 15, 8),
            CSPNetBlock(64, 128, ResNetBlock1, 9, 5),
            CSPNetBlock(128, 128, ResNetBlock2, 1, 1)
        )
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(output_size=1), # globle average pooling
            nn.Flatten(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 16),
            nn.Sigmoid(),
            nn.Linear(16, 1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        y = self.stem(x)
        y = self.body(y)
        y = self.head(y)
        y = torch.clamp(y, max=1.)
        return y

model = Model().cuda()
model

Model(
  (stem): MultiPath(
    (layers1): Sequential(
      (0): Sequential(
        (0): Conv1d(1, 48, kernel_size=(19,), stride=(10,), bias=False)
        (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Conv1d(48, 48, kernel_size=(19,), stride=(10,), bias=False)
        (1): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (layers2): Sequential(
      (0): Sequential(
        (0): Conv1d(1, 16, kernel_size=(9,), stride=(5,), bias=False)
        (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
      (1): Sequential(
        (0): Conv1d(16, 16, kernel_size=(9,), stride=(5,), bias=False)
        (1): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
  

In [4]:
duration = 6*60*60 # 6 hr
SR = 100 # 100 hz
data_len = duration*SR

input_data = torch.ones(1, 1, data_len).cuda() # [batch, channel, row]
input_data.size()

torch.Size([1, 1, 2160000])

In [5]:
writer.add_graph(model, input_data)
writer.close()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs --reload_multifile True

tb_temp_dir = pathlib.Path(tempfile.gettempdir(), '.tensorboard-info')
for tb_temp in tb_temp_dir.glob('*.info'):
    tb_temp.unlink(missing_ok=True)